# Multilingual Customer Support Chatbot (8.4.1 Use Case)

In [ ]:
!pip install langchain chromadb transformers huggingface_hub

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub, HuggingFaceEndpoint
from transformers import pipeline
import os

In [ ]:
faq_content = """Customer Support FAQs:

Q: How can I reset my password?
A: To reset your password, click on \"Forgot Password\" on the login page, then follow the instructions sent to your email.

Q: How do I update my contact information?
A: To update your contact info, navigate to your account settings, then edit your personal details such as phone number or address.

Q: What should I do if my account is locked?
A: If your account is locked, please wait 15 minutes and try again. If the issue persists, contact our support team to unlock your account.
"""
with open("support_faq_en.txt", "w") as f:
    f.write(faq_content)

In [ ]:
loader = TextLoader("support_faq_en.txt")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=150, chunk_overlap=20)
docs = text_splitter.split_documents(documents)
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
vector_store = Chroma.from_documents(docs, embedding_model)

In [ ]:
translator_es_to_en = pipeline("translation", model="Helsinki-NLP/opus-mt-es-en")
translator_en_to_es = pipeline("translation", model="Helsinki-NLP/opus-mt-en-es")
translator_fr_to_en = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en")
translator_en_to_fr = pipeline("translation", model="Helsinki-NLP/opus-mt-en-fr")

In [ ]:
qa_extractive = pipeline("question-answering", model="distilbert-base-multilingual-cased-distilled-squad")
gemma_llm = None
bitnet_llm = None
try:
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = "your_huggingface_api_token"
    bitnet_llm = HuggingFaceHub(repo_id="microsoft/bitnet-b1.58-2B-4T", model_kwargs={"temperature": 0.2, "max_new_tokens": 256})
except:
    pass

In [ ]:
from langchain.llms.base import LLM
from typing import List, Optional
from langchain.schema import Generation

class QAExtractiveLLM(LLM):
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        if "Q:" in prompt and "A:" in prompt:
            parts = prompt.split("Q:")
            context = parts[0]
            question = parts[1].split("A:")[0].strip()
        else:
            context = prompt
            question = prompt
        result = qa_extractive(question=question, context=context)
        return result['answer']
    def _identifying_params(self): return {}
    @property
    def _llm_type(self): return "custom-extractive-qa"

In [ ]:
extractive_chain = RetrievalQA.from_chain_type(llm=QAExtractiveLLM(), chain_type="stuff", retriever=vector_store.as_retriever(search_kwargs={"k": 2}))
generative_chain = None
if bitnet_llm is not None:
    generative_chain = RetrievalQA.from_chain_type(llm=bitnet_llm, chain_type="stuff", retriever=vector_store.as_retriever(search_kwargs={"k": 2}))

In [ ]:
def detect_language(question: str) -> str:
    q_lower = question.lower()
    if "¿" in question or "cómo" in q_lower:
        return "es"
    elif "comment" in q_lower or "ét" in q_lower or "é" in q_lower:
        return "fr"
    else:
        return "en"

def ask_question_extractive(question: str):
    lang = detect_language(question)
    original_question = question
    if lang == "es":
        question = translator_es_to_en(question)[0]['translation_text']
    elif lang == "fr":
        question = translator_fr_to_en(question)[0]['translation_text']
    answer_en = extractive_chain.run(question)
    if lang == "es":
        answer_final = translator_en_to_es(answer_en)[0]['translation_text']
    elif lang == "fr":
        answer_final = translator_en_to_fr(answer_en)[0]['translation_text']
    else:
        answer_final = answer_en
    print(f"Question ({lang}): {original_question}")
    print(f"Answer ({lang}): {answer_final}\n")
    return answer_final

def ask_question_generative(question: str):
    if generative_chain is None:
        print("Generative LLM not available. Using extractive QA instead.")
        return ask_question_extractive(question)
    lang = detect_language(question)
    original_question = question
    if lang == "es":
        question = translator_es_to_en(question)[0]['translation_text']
    elif lang == "fr":
        question = translator_fr_to_en(question)[0]['translation_text']
    answer_en = generative_chain.run(question)
    if lang == "es":
        answer_final = translator_en_to_es(answer_en)[0]['translation_text']
    elif lang == "fr":
        answer_final = translator_en_to_fr(answer_en)[0]['translation_text']
    else:
        answer_final = answer_en
    print(f"Question ({lang}): {original_question}")
    print(f"Answer ({lang}): {answer_final}\n")
    return answer_final

In [ ]:
# Test examples
ask_question_extractive("¿Cómo puedo restablecer mi contraseña?")
ask_question_extractive("Que dois-je faire si mon compte est verrouillé?")
ask_question_generative("¿Cómo puedo restablecer mi contraseña?")